Data Mining. Lab #2
================

[SymPy](https://docs.sympy.org/latest/tutorial/preliminaries.html)

In [1]:
from sympy import *
from itertools import combinations
init_printing()

generate k-subsets from set.

In [2]:
def subsets(S, k):
    return [set(s) for s in combinations(S, k)]

In [3]:
subsets({1, 2, 3}, 2)

And sometimes I want to print a list, each on its own line:

In [4]:
def print_all(iterable):
    for item in iterable:
        print(item)

In [5]:
print_all(subsets({1, 2, 3}, 2))

{1, 2}
{1, 3}
{2, 3}


In association rule mining, we try to discover rules based on item sets.

Our input data is a list of _transactions_. Each transaction contains a set of items (called an _itemset_).
For example, this table represents the set of items bought by a customer in a single transaction:

| TID | Items |
| --- | ----- |
| 1 | Bread, Milk |
| 2 | Bread, Diaper, Beer, Eggs |
| 3 | Milk, Diaper, Beer, Coke |
| 4 | Bread, Milk, Diaper, Beer |
| 4 | Bread, Milk, Diaper, Coke |

In [6]:
T = [
 {'Bread', 'Milk'},
 {'Beer', 'Bread', 'Diaper', 'Eggs'},
 {'Beer', 'Coke', 'Diaper', 'Milk'},
 {'Beer', 'Bread', 'Diaper', 'Milk'},
 {'Bread', 'Coke', 'Diaper', 'Milk'},
]

association rules, such as:

$$ \{\text{Milk}, \text{Bread}\} \Rightarrow \{\text{Eggs}, \text{Coke}\} $$


Itemsets
========

An _itemset_ is simply a set of items, such as $\{ \text{Milk}, \text{Bread}, \text{Eggs} \}$.

The frequency of occurrence of an _itemset_ is called support count.

$$ \sigma(X) = \|\{ x \in T \:|\: X \subseteq x \}\| $$

In [7]:
def support_count(X, T):
    return S(sum(1 for x in T if X <= x))

In [8]:
support_count({'Milk', 'Bread', 'Diaper'}, T)

Support is the _proportion_ of transactions that contain an itemset.

$$ s(X) = \frac{\sigma(X)}{\|\text{T}\|} $$

In [9]:
def support(X, T):
    return support_count(X, T) / len(T)

In [10]:
support({'Milk', 'Bread', 'Diaper'}, T)

If the _support_ is higher than a given ratio, it is called a _frequent itemset_.

Rules
=====

A rule is in form of $X \Rightarrow Y$ where $X$ and $Y$ are itemsets. For example:

$$ \{ \text{Milk}, \text{Diaper} \} \Rightarrow \{ \text{Beer} \} $$

In [11]:
rule = ({'Milk', 'Diaper'}, {'Beer'})

The _support_ of the rule is the fraction of transactions that contain both $X$ and $Y$.

$$ s(X \Rightarrow Y) = s(X \cup Y) $$

In [12]:
def rule_support(xy, T):
    x, y = xy
    return support(x | y, T)

In [13]:
rule_support(rule, T)

The _confidence_ of the rule tells you how many transactions that contains $X$ also contains $Y$ (in form of proportion).

$$ c(X \Rightarrow Y) = \frac{\sigma(X \cup Y)}{\sigma(X)} \left(= \frac{s(X \Rightarrow Y)}{s(X)}\right) $$

In [14]:
def rule_confidence(xy, T):
    x, y = xy
    return support(x | y, T) / support(x, T)

In [15]:
rule_confidence(rule, T)

Association Rule Mining
=======================

In association rule mining, we want to find all rules that has enough _support_ and _confidence_. In other words, we want to find $\{ X \Rightarrow Y \:|\: s(X \Rightarrow Y) > s_{min}, c(X \Rightarrow Y) > c_{min} \}$.

We are using 2-step approach:

- First, finding frequent itemsets with enough support.
    - For example, $\{A, B, C\}$
- Then, generate rules from these itemsets.
    - We can generate rules by finding binary partitions of a given itemset.
    - For example, from $\{A, B, C\}$, we can generate 6 rules:
    - Note that support of these rules are all the same.
    - We then select only the rules with enough confidence.
        - $\{A\} \Rightarrow \{C,B\}$
        - $\{C\} \Rightarrow \{A,B\}$
        - $\{B\} \Rightarrow \{A,C\}$
        - $\{A,C\} \Rightarrow \{B\}$
        - $\{A,B\} \Rightarrow \{C\}$
        - $\{C,B\} \Rightarrow \{A\}$

frequent itemsets supposed to be given

In [16]:
T = [
 {'A', 'B', 'E'},
 {'B', 'D'},
 {'B', 'C'},
 {'A', 'B', 'D'},
 {'A', 'C'},
 {'B', 'C'},
 {'A', 'C'},
 {'A', 'B', 'C', 'E'},
 {'A', 'B', 'C'},
]

cmin = 0.5

Given a Frequent Itemset
------------------------

We can use a naive algorithm:

In [17]:
# frequent itemset `l`
def find_rules(l, T):
    rules = []
    for n in range(1, len(l)):
        for c in subsets(l, n):
            rule = (set(c), l - set(c))
            if rule_confidence(rule, T) >= cmin:
                rules.append(rule)
    return rules

It just tries all "binary partitions" of the frequent itemset `l`, and only emits rules with enough confidence.

In [18]:
[(rule, rule_confidence(rule, T)) for rule in find_rules({'A', 'B', 'E'}, T)]

[(({'E'}, {'A', 'B'}), 1),
 (({'B', 'E'}, {'A'}), 1),
 (({'A', 'E'}, {'B'}), 1),
 (({'A', 'B'}, {'E'}), 1/2)]

Without a Frequent Itemset
--------------------------

Now, recall about having to find the frequent itemsets. What should we do?

For a large dataset, this is impractical. To find all the candidates, we must try all subsets of all items!
If we have $d$ items, the number of subsets become $2^d$.
See how fast it grows!
What can we do to help?

Apriori Principle
-----------------

You know, given that $A \subseteq B$, then $C \subseteq B$ as well if $C \subseteq A$.
It is also obvious that if $A \subseteq B$, $s(A) \geq s(B)$, since every item in $A$ also appear in $B$.

Apriori priciple says:
_"If an itemset is frequent, then all of its subsets must also be frequent."_

From that, we know that if an itemset is _infrequent_, all its supersets are also infrequent.

Assume the example

In [19]:
T = [
  {'A', 'C', 'D'},
  {'B', 'C', 'E'},
  {'A', 'B', 'C', 'E'},
  {'B', 'E'},
  {'A', 'B', 'C', 'E'}
]

smin = 0.4


### Starting Small

So here's our approach.

- First, we find all frequent itemsets of size 1 (called _frequent 1-itemsets_).
- Next, we "prune" itemsets whose support is too low.
- Then, we generate frequent _2_-itemsets from the remaining 1-itemsets.
- Again, we "prune" itemsets whose support is too low.
- Increase the size and repeat.


### Generating Frequent 1-itemsets

First, let's generate frequent 1-itemsets.
Before that, I will create a function to union multiple sets.

In [20]:
def union_all(sets):
    result = set()
    for c in sets:
        result = result | c
    return result

In [21]:
union_all([{1, 2}, {2, 3}])

In [22]:
union_all([])

Now, some code to find frequent 1-itemsets:

In [23]:
def frequent_1(T):
  items = union_all(T)
  return [{item}
    for item in items
      if support({item}, T) >= smin]

In [24]:
L1 = frequent_1(T)
L1

[{'E'}, {'C'}, {'B'}, {'A'}]

As you see, "D" is eliminated from the candidates.
That means any itemset with "D" in it will not be frequent enough.

### Expanding It

Next, we generate frequent 2-itemsets from 1-itemsets.
The easiest way to do it is to put these items together and select 2 items. First, we put them together:

In [25]:
union_all(L1)

{'A', 'B', 'C', 'E'}

In [26]:
C2 = subsets(_, 2)
print_all(C2)

{'E', 'A'}
{'C', 'A'}
{'B', 'A'}
{'E', 'C'}
{'E', 'B'}
{'C', 'B'}


These are the candidate itemsets. But maybe... not all of them are frequent enough.

Now, for each candidate $c$ in `C2`,
we must make sure at all of $c$'s 1-subset is in `L1`.
Why? If one of $c$'s subset (let's call it $s$) is not in `L1`,
it means that that $s$ has already been pruned, because $s$ is not frequent enough.
Since $c$ is a superset of $s$, $c$ will also not be frequent enough.

In [27]:
def good_candidate(c, P):
    for s in subsets(c, len(c) - 1):
        if s not in P: return false
    return True

In [28]:
F2 = [c for c in C2 if good_candidate(c, L1)]
print_all(F2)

{'E', 'A'}
{'C', 'A'}
{'B', 'A'}
{'E', 'C'}
{'E', 'B'}
{'C', 'B'}


Well, it seems that every candidate is a good one. Anyway, now we have the finalists!
For the final round, you might have guessed it:
We simply check the support to see if each item set is frequent enough!

In [29]:
L2 = [f for f in F2 if support(f, T) >= smin]
print_all(L2)

{'E', 'A'}
{'C', 'A'}
{'B', 'A'}
{'E', 'C'}
{'E', 'B'}
{'C', 'B'}


Again, all of them are frequent enough! So, now we have the 2-itemsets.

### Moving On

Now, let's generate `L3`.

In [30]:
def generate_candidates(P, k):
    return subsets(union_all(P), k)

In [31]:
C3 = generate_candidates(L2, 3)
print_all(C3)

{'E', 'C', 'B'}
{'E', 'C', 'A'}
{'A', 'E', 'B'}
{'A', 'C', 'B'}


In [32]:
F3 = [c for c in C3
         if good_candidate(c, L2)]
print_all(F3)

{'E', 'C', 'B'}
{'E', 'C', 'A'}
{'A', 'E', 'B'}
{'A', 'C', 'B'}


In [33]:
L3 = [f for f in F3
         if support(f, T) >= smin]
print_all(L3)

{'E', 'C', 'B'}
{'E', 'C', 'A'}
{'A', 'E', 'B'}
{'A', 'C', 'B'}


### Generalizing It

We can turn the above steps into this function:

In [34]:
def frequent_k(P, k, T):
    ck = generate_candidates(P, k)
    fk = [c for c in ck if good_candidate(c, P)]
    lk = [f for f in fk if support(f, T) >= smin]
    
    return lk

We then use that function to generate `L4`.

In [35]:
L4 = frequent_k(L3, 4, T)
L4

[{'A', 'B', 'C', 'E'}]

Finally, generating `L5` will return no itemsets, which concludes the Apriori algorithm:

In [36]:
L5 = frequent_k(L4, 5, T)
L5

Putting It Together
-------------------

We take all the previous answers to find the frequent itemsets!

In [37]:
L1 + L2 + L3 + L4 + L5

[{'E'},
 {'C'},
 {'B'},
 {'A'},
 {'A', 'E'},
 {'A', 'C'},
 {'A', 'B'},
 {'C', 'E'},
 {'B', 'E'},
 {'B', 'C'},
 {'B', 'C', 'E'},
 {'A', 'C', 'E'},
 {'A', 'B', 'E'},
 {'A', 'B', 'C'},
 {'A', 'B', 'C', 'E'}]

Summing It Up
-------------

Finally, here's the apriori algorithm!

In [38]:
def apriori(T):
    
    P = [{item} for item in union_all(T)]
    max_len = len(P)
    result = P
    
    for k in range(2, max_len + 1):
        
        P = frequent_k(P, k, T)
        result += P
        
        if not len(P): break
    
    return result

In [39]:
L = apriori(T)
L

[{'E'},
 {'D'},
 {'C'},
 {'B'},
 {'A'},
 {'A', 'E'},
 {'C', 'E'},
 {'B', 'E'},
 {'A', 'C'},
 {'A', 'B'},
 {'B', 'C'},
 {'B', 'C', 'E'},
 {'A', 'C', 'E'},
 {'A', 'B', 'E'},
 {'A', 'B', 'C'},
 {'A', 'B', 'C', 'E'}]

For each frequent itemsets, we generate rules from it.

In [40]:
cmin = 0.75

[rule
  for itemset in L
    for rule in find_rules(itemset, T)]

[({'E'}, {'C'}),
 ({'C'}, {'E'}),
 ({'E'}, {'B'}),
 ({'B'}, {'E'}),
 ({'C'}, {'A'}),
 ({'A'}, {'C'}),
 ({'C'}, {'B'}),
 ({'B'}, {'C'}),
 ({'E'}, {'B', 'C'}),
 ({'C'}, {'B', 'E'}),
 ({'B'}, {'C', 'E'}),
 ({'C', 'E'}, {'B'}),
 ({'B', 'E'}, {'C'}),
 ({'B', 'C'}, {'E'}),
 ({'A', 'E'}, {'C'}),
 ({'A', 'E'}, {'B'}),
 ({'A', 'B'}, {'E'}),
 ({'A', 'B'}, {'C'}),
 ({'A', 'E'}, {'B', 'C'}),
 ({'A', 'B'}, {'C', 'E'}),
 ({'A', 'C', 'E'}, {'B'}),
 ({'A', 'B', 'E'}, {'C'}),
 ({'A', 'B', 'C'}, {'E'})]

The Lift
--------

How can you be sure that there really is a correlation between the itemset $X$ and $Y$?

Take an example from the slides:

- 90% of customers buy coffee.
- 25% of customers buy tea.
- 20% of customers buy both.

After filling Venn diagram, here's our transactions:

In [41]:
T = (
  20 * [{'coffee', 'tea'}] +
  70 * [{'coffee'}] +
   5 * [{'tea'}] +
   5 * [set()]
)

Given $\{\text{coffee}, \text{tea}\}$ is a frequent itemset,
let's mine some rules!

In [42]:
rules = find_rules({'coffee', 'tea'}, T)
rules

[({'tea'}, {'coffee'})]

Here, we mined the rule $\{\text{tea}\} \Rightarrow \{\text{coffee}\}$.
How confident we are?

In [43]:
rule = ({'tea'}, {'coffee'})
rule_confidence(rule, T)

We found that __80% of customers that buy tea also buys coffee__.
We're highly confident, at 80 percent!
But is 80% good?

Does it really mean that the customer buys coffee _because_ they buys tea?
To find out, let's remove the condition.
Let's see how many people buys coffee "no matter what:"

In [44]:
rule = (set(), {'coffee'})
rule_confidence(rule, T)

This is called the unconditional, _expected confidence_.
As you see, people who buy tea are actually less likely to buy coffee.
This is a _negative correlation_.

To quantify this correlation, we use a measure called "lift:"

$$ L(X \Rightarrow Y) = \frac{c(X \Rightarrow Y)}{c(\varnothing \Rightarrow Y)}$$

In [45]:
def rule_lift(xy, T):
    return rule_confidence(xy, T) \
        / rule_confidence((set(), xy[1]), T)

In [46]:
rule = ({'tea'}, {'coffee'})
rule_lift(rule, T)

The "lift" measure tells us the correlation of the rule.

- $L > 1 \Rightarrow$ positive correlation
- $L = 1 \Rightarrow$ independence
- $L < 1 \Rightarrow$ negative correlation

And this concludes this notebook regarding association rules and apriori algorithm.

## Tasks


[The Intuition Behind the Apriori Algorithm](https://medium.com/weekly-data-science/the-intuition-behind-the-apriori-algorithm-4efe312ccc3c)

[Hash Functions for Data Mining](https://medium.com/weekly-data-science/hash-functions-for-data-mining-55e9c7760703)

## Question 1
Suppose we have transactions that satisfy the following assumptions: 
- s, the support threshold, is 10,000.
- There are one million items, which are represented by the integers 0,1,...,999999.
- There are N frequent items, that is, items that occur 10,000 times or more.
- There are one million pairs that occur 10,000 times or more.
- There are 2M pairs that occur exactly once. M of these pairs consist of two frequent items, the other M each have at least one nonfrequent item.
- No other pairs occur at all.
- Integers are always represented by 4 bytes.

Suppose we run the a-priori algorithm to find frequent pairs and can choose on the second pass between the triangular-matrix method for counting candidate pairs (a triangular array $count[i][j]$ that holds an integer count for each pair of items (i, j) where i < j) and a hash table of item-item-count triples. Neglect in the first case the space needed to translate between original item numbers and numbers for the frequent items, and in the second case neglect the space needed for the hash table. Assume that item numbers and counts are always 4-byte integers. 

As a function of N and M, what is the minimum number of bytes of main memory needed to execute the a-priori algorithm on this data? Demonstrate that you have the correct formula by selecting, from the choices below, the triple consisting of values for N, M, and the (approximate, i.e., to within 10%) minumum number of bytes of main memory, S, needed for the a-priori algorithm to execute with this data.

In [47]:
import os
import sys

# N = 100,000; M = 50,000,000; S = 5,000,000,000
# N = 40,000; M = 60,000,000; S = 3,200,000,000
# N = 50,000; M = 80,000,000; S = 1,500,000,000
# N = 100,000; M = 100,000,000; S = 1,200,000,000
soln = [[100000, 50000000, 5000000000],
        [40000, 60000000, 3200000000],
        [50000, 80000000, 1500000000],
        [100000, 100000000, 1200000000]]

In [48]:
def count_memory(n, m):
    triangular = 2 * n * (n-1)
    triples_list = 12 * m
    
    return min(triangular, triples_list)


In [49]:
for n, m, s in soln:
    s_c = count_memory(n, m)
    if s_c * 1.1 >= s >= s_c * .9:
        print('N =', n, '\b; M =', m, '\b; S =', s) 

N = 100000 ; M = 100000000 ; S = 1200000000


**Answer**: N = 100,000; M = 100,000,000; S = 1,200,000,000

## Question 2
Imagine there are 100 baskets, numbered 1,2,...,100, and 100 items, similarly numbered. Item i is in basket j if and only if i divides j evenly. For example, basket 24 is the set of items {1,2,3,4,6,8,12,24}. Describe all the association rules that have 100% confidence. Which of the following rules has 100% confidence?

In [50]:
baskets = range(1,101)
items = range(1,101)

# Create transactions
transactions = []

for i in baskets:
    basket = []
    for item in items:
        if i % item == 0:
            basket.append(item)
    transactions.append(basket)

In [51]:
# Brute force implementation
#def confidence(num, denom):
#    c_num, c_denom = 0, 0
#    num, denom = set(num), set(denom)
#    
#    for t in transactions:
#        t = set(t)
#        c_num += 1 if num <= t else 0
#        c_denom += 1 if denom <= t else 0
#        if num <= t:
#            print('n', t)
#        if denom <= t:
#            print('d', t)
#            
#    return c_num / c_denom

from functools import reduce
from math import gcd

def lcms(numbers):
    return reduce(lambda a, b: a * b // gcd(a, b), numbers)


def confidence(num, denom):
    return (100 // lcms(num)) / (100 // lcms(denom))

print("""
Describe all the association rules that have 100% confidence:

The rule X -> Y has 100% confidence when X U Y and X lays in the same baskets.

The itemset lays in some basket N only if N is a multiple of least common multiple of basket's items.
Thus, the support_count(X) = 100 // LCM(X).
""")


Describe all the association rules that have 100% confidence:

The rule X -> Y has 100% confidence when X U Y and X lays in the same baskets.

The itemset lays in some basket N only if N is a multiple of least common multiple of basket's items.
Thus, the support_count(X) = 100 // LCM(X).



In [52]:
print("{1,2}-> 4,Condidence = %d"%(confidence([1,2,4],[1,2])))
print("{1}-> 2,Condidence = %d"%(confidence([1,2],[1])))
print("{1,4,7}-> 14,Condidence = %d"%(confidence([1,4,7,14],[1,4,7])))
print("{1,3,6}-> 12,Condidence = %d"%(confidence([1,3,6,12],[1,3,6])))
print("{4,6}-> 12,Condidence = %d"%(confidence([4,6,12],[4,6])))
print("{8,12}-> 96,Condidence = %d"%(confidence([8,12,96],[8,12])))
print("{4,6}-> 24,Condidence = %d"%(confidence([4,6,24],[4,6])))
print("{1,3,6}-> 12,Condidence = %d"%(confidence([1,3,6,12],[1,3,6])))

{1,2}-> 4,Condidence = 0
{1}-> 2,Condidence = 0
{1,4,7}-> 14,Condidence = 1
{1,3,6}-> 12,Condidence = 0
{4,6}-> 12,Condidence = 1
{8,12}-> 96,Condidence = 0
{4,6}-> 24,Condidence = 0
{1,3,6}-> 12,Condidence = 0
